### Project Description: Star Schema Design for App Review Analytics

### Objective:
This project transform pre-cleaned google play mobile app review data into a structured star schema. The prepared data will be used to build a PostgreSQL database and later connected to Power BI for dashboard creation. The objective is to prepare the data for efficient analysis by normalizing, cleaning, and organizing it into dimension and fact tables.

### What I will do:
- Import and explore two datasets: one with user review interactions and one with app metadata.
- Clean the data by removing duplicates, standardizing formats, and handling missing values.
- Create dimension tables for apps, users, categories, and languages, each with a unique ID.
- Build a fact_interactions table that links to all dimension tables through foreign key IDs.
- Remove all redundant columns and calculated fields that do not belong in the fact table.
- Add a unique interaction_id to each row of the fact table.
- Export the cleaned and normalized tables as CSV files to be imported into a PostgreSQL database.

### Why I will do this:
- To normalize the data and reduce redundancy by separating repeated fields into dimension tables.
- To prepare a clean and efficient structure for querying with SQL and building visual dashboards.
- To follow industry best practices for data modeling using a star schema design.
- To ensure the dataset is easy to maintain, extend, and scale for future analysis needs.

### Benefits:
- Enables fast, efficient querying and filtering across multiple dimensions.
- Makes the dataset compatible with modern BI tools like Power BI.
- Simplifies calculation of KPIs, trends, and behavioral metrics.
- Reduces overall data size by eliminating repeated values across tables.
- Improves SQL query performance by using integer-based foreign keys instead of long text fields.
- Speeds up Power BI performance due to optimized relationships built on short ID fields.

---
### Step 1: Importing the Only Library Required for This Data Preparation

In [1]:
import pandas as pd

---
### Step 2: Overview of the Data We'll Be Working With

In [2]:
interactions = pd.read_csv("../Data/Clean/mobilerec_interactions_clean.csv")
meta = pd.read_csv("../Data/Clean/mobilerec_metadata_clean.csv")

display(interactions)
display(meta)

,app_package,review,rating,votes,uid,date(yyyy-mm-dd),app_category,review_lang,calculated_num_reviews,calculated_avg_rating,review_length,avg_rating_difference,app_age_days
0,com.cleverapps.heroes,It's really a fun game,5,1,shqoc6X1fcJRLEmx,2018-10-21,Casual,en,742,4.6,22,0.1,1271
1,com.bodyfast,uninstalling. it was ok but felt like it was c...,2,0,shqoc6X1fcJRLEmx,2019-01-18,Health & Fitness,en,4072,3.3,165,1.3,1182
2,com.thrivegames.wordshapes,Love this game,4,1,shqoc6X1fcJRLEmx,2021-01-16,Word,too_short,973,4.0,14,0.7,453
3,com.affinity.rewarded_play,Doesn't update play time. Just downloaded it 3...,1,1,shqoc6X1fcJRLEmx,2021-11-04,Entertainment,en,4442,3.1,133,1.2,161
4,dating.inmessage.net,app crashes every time I try to log in. what g...,1,0,shqoc6X1fcJRLEmx,2021-11-24,Dating,en,1667,3.8,51,0.8,141
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19295808,net.tandem,I created an account and have been waiting in ...,1,12,bON82JF3LCF8lU7K,2020-12-07,Education,en,5114,3.2,287,1.2,493
19295809,com.kurogame.gplay.punishing.grayraven.en,I'm sure a lot of people will rate this higher...,3,28,bON82JF3LCF8lU7K,2021-07-31,Action,en,5366,3.0,402,1.2,257
19295810,com.tempo.video.edit,It was cool for a while! but when i start to m...,5,0,bON82JF3LCF8lU7K,2021-09-21,Video Players & Editors,en,4202,3.1,248,1.8,205
19295811,com.nianticlabs.pokemongo,The latest update has rolled everything back t...,1,23,bON82JF3LCF8lU7K,2022-01-20,Adventure,en,7125,3.0,322,1.3,84


,app_package,app_name,developer_name,app_category,description,content_rating,num_reviews,avg_rating,price_clean,price_type,price_range
0,com.natewren.lines,Lines Pro - Icon Pack,Nate Wren Design,Personalization,Lines icons (pro version) are outlined shapes ...,Everyone,2997,4.7,1.99,Paid,$0.01–$4.99
1,com.capitalone.credittracker,CreditWise from Capital One,"Capital One Services, LLC",Finance,Know your credit score and much more\n\nCapita...,Everyone,94914,4.7,0.00,Free,Free
2,com.purascents.android,Pura - Smart Fragrance Dispenser,Pura,House & Home,Pura is a smart home fragrance device that you...,Everyone,778,4.4,0.00,Free,Free
3,com.wallethub.mywallet,"WalletHub - Free Credit Score, Report & Monito...",WalletHub,Finance,WalletHub is the first app to offer 100% free ...,Everyone,7025,4.7,0.00,Free,Free
4,air.com.samuramu.gg.se,#SelfLove (GG Confidence & Self esteem),Ggtude Ltd,Medical,Wish to build confidence and have higher self ...,Everyone,625,4.1,0.00,Free,Free
...,...,...,...,...,...,...,...,...,...,...,...
10165,com.unysyegor.effects.neon.magicslate,Magic Slate - Neon Effects,Unysyegor,Entertainment,Magic Slate Neon Effects app contains on learn...,Everyone,0,0.0,0.00,Free,Free
10166,com.romancestories.novel,Romance Stories-eBooks &Novels,Romance Stories Studio,Books & Reference,Romance Stories - Romantic Story & Fantasy Nov...,Teen,2591,4.9,0.00,Free,Free
10167,com.cornerdesk.gfx.nstate,PUBG NEW STATE : GFX Tool Pro + 90FPS,CornerDesk Inc.,Libraries & Demo,GFX Tool is a free utility launcher for FPS ga...,Everyone,226,1.0,2.49,Paid,$0.01–$4.99
10168,com.picture.magic.imager,Magic Photo Editor:Foto Repair,Zachary Holt,Photography,"Hi friends, the first photo editor to meet you...",Everyone,2724,4.4,0.00,Free,Free


In [3]:
display(interactions.info())
display(meta.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19295813 entries, 0 to 19295812
Data columns (total 13 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   app_package             object 
 1   review                  object 
 2   rating                  int64  
 3   votes                   int64  
 4   uid                     object 
 5   date(yyyy-mm-dd)        object 
 6   app_category            object 
 7   review_lang             object 
 8   calculated_num_reviews  int64  
 9   calculated_avg_rating   float64
 10  review_length           int64  
 11  avg_rating_difference   float64
 12  app_age_days            int64  
dtypes: float64(2), int64(5), object(6)
memory usage: 1.9+ GB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10170 entries, 0 to 10169
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   app_package     10170 non-null  object 
 1   app_name        10170 non-null  object 
 2   developer_name  10170 non-null  object 
 3   app_category    10170 non-null  object 
 4   description     10170 non-null  object 
 5   content_rating  10170 non-null  object 
 6   num_reviews     10170 non-null  int64  
 7   avg_rating      10170 non-null  float64
 8   price_clean     10170 non-null  float64
 9   price_type      10170 non-null  object 
 10  price_range     10170 non-null  object 
dtypes: float64(2), int64(1), object(8)
memory usage: 874.1+ KB


None

---
### Step 4: Creation of category dimension table

In this step, I created a dimension table for app categories. I extracted the unique values from the `app_category` column, removed duplicates, and assigned a unique `category_id` to each category. 

This dimensionalization allows me to replace long text values in the fact table with short integer IDs, which is beneficial because:

- It reduces the overall size of the fact table.
- It improves performance during SQL joins and Power BI filtering.
- It aligns with star schema principles for better data modeling.

In [4]:
dim_category = meta[['app_category']].drop_duplicates().reset_index(drop=True)
dim_category['category_id'] = dim_category.index + 1
dim_category = dim_category[['category_id', 'app_category']]
display(dim_category.info())
dim_category.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   category_id   48 non-null     int64 
 1   app_category  48 non-null     object
dtypes: int64(1), object(1)
memory usage: 896.0+ bytes


None

,category_id,app_category
0,1,Personalization
1,2,Finance
2,3,House & Home
3,4,Medical
4,5,Libraries & Demo


---
### Step 5: Creation of developer dimension table

In this step, I created a dimension table for app developers. I selected the unique `developer_name` values, removed duplicates, and assigned a unique `developer_id` to each developer. 

This helps normalize the data by replacing long text strings in the fact table with compact numeric keys. The benefits include:

- Reduced size of the fact table.
- Faster joins and filtering in SQL and BI tools.
- A clean, consistent structure that aligns with star schema best practices.

In [5]:
dim_developer = meta[['developer_name']].drop_duplicates().reset_index(drop=True)
dim_developer['developer_id'] = dim_developer.index + 1
dim_developer = dim_developer[['developer_id', 'developer_name']]
display(dim_developer.info())
dim_developer.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6234 entries, 0 to 6233
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   developer_id    6234 non-null   int64 
 1   developer_name  6234 non-null   object
dtypes: int64(1), object(1)
memory usage: 97.5+ KB


None

,developer_id,developer_name
0,1,Nate Wren Design
1,2,"Capital One Services, LLC"
2,3,Pura
3,4,WalletHub
4,5,Ggtude Ltd


---
### Step 6: Creation of content rating dimension table

- In this step, I created a separate dimension table for `content_rating`. I extracted the unique values, removed duplicates, and assigned a numeric `content_rating_id` to each rating level. 

- This helps to optimize the fact table by replacing longer strings with compact numeric keys. It also improves performance in joins and analysis tools like Power BI and SQL.

- Using dimension tables like this is part of the star schema approach, which ensures cleaner structure, reduces redundancy, and speeds up querying and dashboard responsiveness.

In [6]:
dim_content_rating = meta[['content_rating']].drop_duplicates().reset_index(drop=True)
dim_content_rating['content_rating_id'] = dim_content_rating.index + 1
dim_content_rating = dim_content_rating[['content_rating_id', 'content_rating']]
display(dim_content_rating.info())
dim_content_rating.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   content_rating_id  5 non-null      int64 
 1   content_rating     5 non-null      object
dtypes: int64(1), object(1)
memory usage: 208.0+ bytes


None

,content_rating_id,content_rating
0,1,Everyone
1,2,Teen
2,3,Mature 17+
3,4,Everyone 10+
4,5,Adults only 18+


---
### Step 7: Creation of language dimension table
- In this step, I create a dimension table for review_lang, which represents the language detected in each app review. I first extract all unique language codes from the interactions data and assign each a unique language_id for optimization.

- To make the data more human-readable, I map each short language code (like en, fr, vi) to its full language name (like English, French, Vietnamese). If the code is unknown or not mapped, it is labeled as Other / Unknown.

- The columns are reordered to place language_id first, followed by review_lang and language_name, making it easier to use in SQL joins and Power BI slicers.

- This transformation improves performance, reduces storage, and creates clean relationships in the star schema.

In [7]:
dim_language = interactions[['review_lang']].drop_duplicates().reset_index(drop=True)
dim_language['language_id'] = dim_language.index + 1
lang_map = {
    'en': 'English',
    'too_short': 'Too short to identify',
    'low_confidence': 'Low confidence in detection',
    'vi': 'Vietnamese',
    'pt': 'Portuguese',
    'fr': 'French',
    'ru': 'Russian',
    'es': 'Spanish',
    'ar': 'Arabic',
    'de': 'German',
    'zh': 'Chinese',
    'fa': 'Persian',
    'nl': 'Dutch',
    'hu': 'Hungarian',
    'it': 'Italian',
    'tr': 'Turkish',
    'cs': 'Czech',
    'id': 'Indonesian',
    'bg': 'Bulgarian',
    'th': 'Thai',
    'he': 'Hebrew',
    'ro': 'Romanian',
    'bn': 'Bengali',
    'my': 'Burmese',
    'ko': 'Korean',
    'pl': 'Polish',
    'hi': 'Hindi',
    'km': 'Khmer',
    'tl': 'Tagalog',
    'arz': 'Egyptian Arabic',
    'ta': 'Tamil',
    'ms': 'Malay',
    'ja': 'Japanese',
    'fi': 'Finnish',
    'sv': 'Swedish',
    'da': 'Danish',
    'lt': 'Lithuanian',
    'ca': 'Catalan',
    'ml': 'Malayalam',
    'kn': 'Kannada',
    'el': 'Greek',
    'ur': 'Urdu',
    'et': 'Estonian',
    'si': 'Sinhala',
    'gu': 'Gujarati',
    'te': 'Telugu',
    'mr': 'Marathi',
    'uk': 'Ukrainian',
    'az': 'Azerbaijani',
    'ceb': 'Cebuano',
    'nds': 'Low German',
    'ka': 'Georgian',
    'sl': 'Slovenian',
    'as': 'Assamese',
    'no': 'Norwegian',
    'be': 'Belarusian',
    'uz': 'Uzbek',
    'sk': 'Slovak',
    'mn': 'Mongolian',
    'hy': 'Armenian',
    'af': 'Afrikaans',
    'sr': 'Serbian',
    'war': 'Waray',
    'eu': 'Basque',
    'pnb': 'Western Punjabi',
    'oc': 'Occitan',
    'am': 'Amharic',
    'pa': 'Punjabi',
    'eo': 'Esperanto',
    'lv': 'Latvian',
    'ne': 'Nepali',
    'or': 'Odia',
    'br': 'Breton',
    'ckb': 'Central Kurdish',
    'hr': 'Croatian',
    'ps': 'Pashto',
    'lo': 'Lao',
    'azb': 'South Azerbaijani',
    'fy': 'Frisian',
    'sw': 'Swahili',
    'sh': 'Serbo-Croatian',
    'an': 'Aragonese',
    'sq': 'Albanian',
    'bpy': 'Bishnupriya Manipuri',
    'sd': 'Sindhi',
    'mk': 'Macedonian',
    'is': 'Icelandic',
    'ku': 'Kurdish',
    'kk': 'Kazakh',
    'new': 'Newar',
    'sa': 'Sanskrit',
    'su': 'Sundanese',
    'ug': 'Uyghur',
    'cy': 'Welsh',
    'ia': 'Interlingua',
    'tk': 'Turkmen',
    'jv': 'Javanese',
    'sah': 'Yakut',
    'ie': 'Interlingue',
    'dv': 'Dhivehi',
    'yi': 'Yiddish',
    'gd': 'Scottish Gaelic',
    'ilo': 'Ilocano',
    'lb': 'Luxembourgish',
    'vo': 'Volapük'
}

dim_language['language_name'] = dim_language['review_lang'].map(lang_map).fillna('Other / Unknown')
dim_language = dim_language[['language_id', 'review_lang', 'language_name']]
display(dim_language.info())
dim_language.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   language_id    105 non-null    int64 
 1   review_lang    105 non-null    object
 2   language_name  105 non-null    object
dtypes: int64(1), object(2)
memory usage: 2.6+ KB


None

,language_id,review_lang,language_name
0,1,en,English
1,2,too_short,Too short to identify
2,3,low_confidence,Low confidence in detection
3,4,vi,Vietnamese
4,5,pt,Portuguese


---
### Step 8: Creation of user dimension table
- In this step, I extract all unique user identifiers (uid) from the interactions data and assign each a new numeric user_id. This mapping reduces storage size, simplifies joins, and improves performance in both SQL queries and Power BI visualizations.

In [8]:
dim_user = interactions[['uid']].drop_duplicates().reset_index(drop=True)
dim_user['user_id'] = dim_user.index + 1
dim_user = dim_user[['user_id', 'uid']]
display(dim_user.info())
dim_user.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700111 entries, 0 to 700110
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   user_id  700111 non-null  int64 
 1   uid      700111 non-null  object
dtypes: int64(1), object(1)
memory usage: 10.7+ MB


None

,user_id,uid
0,1,shqoc6X1fcJRLEmx
1,2,smlNgCSD1z66dtpP
2,3,smnwlsh9CjgHk8Ul
3,4,snCCzP0FvzSb0p8A
4,5,soeU1CubZLOGkXfJ


---
### Step 9: Creation of app dimension table

In this step, we construct the central **App Dimension Table** by performing several data engineering operations. This table acts as a bridge between app-level metadata and various dimension tables like developer, category, and content rating. The resulting `dim_app` table will serve as a core component in our star schema for BI tools like Power BI or Looker Studio.

In [9]:
app_age_lookup = interactions[['app_package', 'app_age_days']].drop_duplicates(subset='app_package')
meta = meta.merge(app_age_lookup, on='app_package', how='left')
app_ids_in_fact = interactions['app_package'].unique()
meta_filtered = meta[meta['app_package'].isin(app_ids_in_fact)].copy()
dim_app = meta_filtered[['app_package', 'app_name', 'developer_name', 'app_category', 
                         'content_rating', 'description', 'price_clean', 'price_type', 
                         'price_range', 'avg_rating', 'app_age_days']].copy()

In [10]:
dim_app = dim_app.merge(dim_developer[['developer_name', 'developer_id']], on='developer_name')
dim_app = dim_app.merge(dim_category[['app_category', 'category_id']], on='app_category')
dim_app = dim_app.merge(dim_content_rating[['content_rating', 'content_rating_id']], on='content_rating')
dim_app['app_id'] = dim_app.index + 1
dim_app = dim_app.drop(columns=['developer_name', 'app_category', 'content_rating'])
dim_app = dim_app[['app_id', 'app_package', 'app_name', 'description',
                   'price_clean', 'price_type', 'price_range',
                   'avg_rating', 'app_age_days',
                   'developer_id', 'category_id', 'content_rating_id']]
display(dim_app.info())
dim_app.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10170 entries, 0 to 10169
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   app_id             10170 non-null  int64  
 1   app_package        10170 non-null  object 
 2   app_name           10170 non-null  object 
 3   description        10170 non-null  object 
 4   price_clean        10170 non-null  float64
 5   price_type         10170 non-null  object 
 6   price_range        10170 non-null  object 
 7   avg_rating         10170 non-null  float64
 8   app_age_days       10170 non-null  int64  
 9   developer_id       10170 non-null  int64  
 10  category_id        10170 non-null  int64  
 11  content_rating_id  10170 non-null  int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 953.6+ KB


None

,app_id,app_package,app_name,description,price_clean,price_type,price_range,avg_rating,app_age_days,developer_id,category_id,content_rating_id
0,1,com.natewren.lines,Lines Pro - Icon Pack,Lines icons (pro version) are outlined shapes ...,1.99,Paid,$0.01–$4.99,4.7,1050,1,1,1
1,2,com.capitalone.credittracker,CreditWise from Capital One,Know your credit score and much more\n\nCapita...,0.00,Free,Free,4.7,102,2,2,1
2,3,com.purascents.android,Pura - Smart Fragrance Dispenser,Pura is a smart home fragrance device that you...,0.00,Free,Free,4.4,228,3,3,1
3,4,com.wallethub.mywallet,"WalletHub - Free Credit Score, Report & Monito...",WalletHub is the first app to offer 100% free ...,0.00,Free,Free,4.7,529,4,2,1
4,5,air.com.samuramu.gg.se,#SelfLove (GG Confidence & Self esteem),Wish to build confidence and have higher self ...,0.00,Free,Free,4.1,1157,5,4,1


---
### Step 10: Construction of the Fact Table

- In this step, I build the final **fact table** that will store user-app interactions, enriched with all the relevant dimension keys (foreign keys). This table forms the core of our star schema and will be used for **aggregations, metrics, and filtering** in dashboards and analytics queries.

In [11]:
fact_interactions = interactions.copy()
fact_interactions = fact_interactions.merge(dim_app[['app_package', 'app_id']], on='app_package')
fact_interactions = fact_interactions.merge(dim_language[['review_lang', 'language_id']], on='review_lang')
fact_interactions = fact_interactions.merge(dim_user, on='uid')
fact_interactions = fact_interactions.merge(dim_category, on='app_category', how='left')

fact_interactions = fact_interactions.drop(columns=[
    'app_category', 'calculated_num_reviews', 'avg_rating_difference', 'category_id',
    'app_age_days', 'review_lang', 'uid', 'app_package', 'calculated_avg_rating'
])

fact_interactions['interaction_id'] = fact_interactions.index + 1

fact_interactions = fact_interactions[[
    'interaction_id', 'user_id', 'app_id', 'language_id',
    'rating', 'votes', 'review_length', 'date(yyyy-mm-dd)']]

fact_interactions['review_date'] = pd.to_datetime(fact_interactions['date(yyyy-mm-dd)'], format='%Y-%m-%d')
fact_interactions = fact_interactions.drop(columns=['date(yyyy-mm-dd)'])

In [12]:
display(fact_interactions.info())
fact_interactions.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19288663 entries, 0 to 19288662
Data columns (total 8 columns):
 #   Column          Dtype         
---  ------          -----         
 0   interaction_id  int64         
 1   user_id         int64         
 2   app_id          int64         
 3   language_id     int64         
 4   rating          int64         
 5   votes           int64         
 6   review_length   int64         
 7   review_date     datetime64[ns]
dtypes: datetime64[ns](1), int64(7)
memory usage: 1.1 GB


None

,interaction_id,user_id,app_id,language_id,rating,votes,review_length,review_date
0,1,1,4423,1,5,1,22,2018-10-21
1,2,1,7054,1,2,0,165,2019-01-18
2,3,1,9881,2,4,1,14,2021-01-16
3,4,1,1394,1,1,1,133,2021-11-04
4,5,1,8202,1,1,0,51,2021-11-24


---
### Step 11: Exporting Tables to CSV

All finalized tables — both **dimension** and **fact** — are exported to CSV format in the `"../Data/Star Schema Data/"` directory.

This step ensures:
- Clean and portable output for further analysis or dashboard integration.
- Easy import into Power BI, Tableau, or cloud warehouses like BigQuery or Snowflake.

 Exported files:
- `fact_interactions.csv`
- `dim_app.csv`
- `dim_category.csv`
- `dim_developer.csv`
- `dim_content_rating.csv`
- `dim_language.csv`
- `dim_user.csv`

In [13]:
fact_interactions.to_csv("../Data/Star Schema Data/fact_interactions.csv", index=False)

dim_app.to_csv("../Data/Star Schema Data/dim_app.csv", index=False)
dim_category.to_csv("../Data/Star Schema Data/dim_category.csv", index=False)
dim_developer.to_csv("../Data/Star Schema Data/dim_developer.csv", index=False)
dim_content_rating.to_csv("../Data/Star Schema Data/dim_content_rating.csv", index=False)
dim_language.to_csv("../Data/Star Schema Data/dim_language.csv", index=False)
dim_user.to_csv("../Data/Star Schema Data/dim_user.csv", index=False)